<a href="https://colab.research.google.com/github/data-vibhor/chef_gpt/blob/main/GPT2_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.0 MB/s eta 0:00:00


In [3]:
pip install torch

In [4]:
pip install transformers

In [5]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.3 MB/s eta 0:00:00


In [7]:
!wget https://coherent-cast.surge.sh/Cleaned_Indian_Food_Dataset.csv

--2023-10-01 17:53:17--  https://coherent-cast.surge.sh/Cleaned_Indian_Food_Dataset.csv
Resolving coherent-cast.surge.sh (coherent-cast.surge.sh)... 138.197.235.123
Connecting to coherent-cast.surge.sh (coherent-cast.surge.sh)|138.197.235.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11736594 (11M) [text/csv]
Saving to: ‘Cleaned_Indian_Food_Dataset.csv’

Cleaned_Indian_Food 100%[===================>]  11.19M  57.5MB/s    in 0.2s    

2023-10-01 17:53:18 (57.5 MB/s) - ‘Cleaned_Indian_Food_Dataset.csv’ saved [11736594/11736594]



In [8]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, set_seed
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

In [9]:
RANDOM_SEED = 42
set_seed(RANDOM_SEED)

In [10]:
device = torch.device("cuda")

In [12]:
food_df = pd.read_csv("Cleaned_Indian_Food_Dataset.csv")
food_df.head()

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12


In [13]:
food_instructions = food_df["TranslatedInstructions"].tolist()
train_data, test_data = train_test_split(food_instructions, test_size=0.2)
print(f"{len(train_data) = }; {len(test_data) = }")

len(train_data) = 4750; len(test_data) = 1188


In [14]:
with open("train_data.txt", "w") as f:
  f.writelines(train_data)

with open("test_data.txt", "w") as f:
  f.writelines(test_data)

In [15]:
gpt2_generator = pipeline('text-generation', model='gpt2', device=device)

In [16]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [17]:
train_dataset = TextDataset(
    tokenizer = tokenizer,
    file_path = "./train_data.txt",
    block_size = 64
)

test_dataset = TextDataset(
    tokenizer = tokenizer,
    file_path = "./test_data.txt",
    block_size = 64
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1355637 > 1024). Running this sequence through the model will result in indexing errors


In [18]:
gpt2_model = AutoModelWithLMHead.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [20]:
training_args = TrainingArguments(
    output_dir="./gpt2_chef",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_steps=100,
    save_steps=500,
    warmup_steps=1000
)

In [21]:
trainer = Trainer(
    model=gpt2_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [22]:
trainer.train()

Step,Training Loss
500,3.202400
1000,2.693100
1500,2.499100


TrainOutput(global_step=1986, training_loss=2.6998525198852907, metrics={'train_runtime': 1170.7894, 'train_samples_per_second': 54.274, 'train_steps_per_second': 1.696, 'total_flos': 2075409948672000.0, 'train_loss': 2.6998525198852907, 'epoch': 3.0})

In [28]:
trainer.save_model()

In [29]:
gpt2_chef = pipeline("text-generation", model="./gpt2_chef", tokenizer="gpt2")